# Filtering Ofcom Wireless Telegraphy Register

The Wireless Telegraphy Register (WTR) is a publicly available database containing all of the UK's licencing information. The WTR contains information such as licence number, issue date, coordinates and licence holder among many others. The scope of the WTR is much larger than is required to visualise released spectrum in for Wireless Broad Access and will require extensive filtering and conditional formatting.

The WTR contains every licence, over all of the UK's spectrum. Filters must be developed to isolate mid-band licences used exclusively for WBA and must exclude extraneous information such as antenna specifications and device location.

## WTR Complications

Unlike the RRL, the required data exists in a single CSV file, however, there aren't any Ofcom classifications for WBA licences which means that filter criteria need developed very carefully to ensure all non-WBA licences are excluded.

In [16]:
import pandas as pd
import numpy as np

wtr = pd.read_csv('WTR.csv')

# Remove unnecessary columns
wtr = wtr[['Licence Number', 
    'Frequency (Hz)', 
    'Station Type', 
    'Channel Width (Hz)', 
    'Product Description']]

# Convert Hz to MHz
wtr['Frequency (Hz)'] = (wtr['Frequency (Hz)'] / 1000000).round(5)
wtr['Channel Width (Hz)'] = (wtr['Channel Width (Hz)'] / 1000000).round(5)

# Change column names
wtr.rename(columns={'Frequency (Hz)' : 'FREQUENCY',
    'Channel Width (Hz)' : 'BW',
    'Licence Number' : 'LICENCE_NO',
    'Station Type' : 'STATION_TYPE',
    'Product Description' : 'CATEGORY'}, inplace=True)

# Isolate mid-band frequencies
wtr = wtr[wtr['FREQUENCY'] < 7000]
wtr.sort_values(by=['LICENCE_NO','FREQUENCY'], inplace=True)
wtr.reset_index(drop=True, inplace=True)

# Save dataset to CSV
wtr.to_csv('Ofcom Datasets/spectrumLicences.csv', index=False)
wtr

,LICENCE_NO,FREQUENCY,STATION_TYPE,BW,CATEGORY
0,0000036/2,165.58750,T,0.0125,BR Tech Assigned
1,0000036/2,165.58750,R,0.0125,BR Tech Assigned
2,0000036/2,170.38750,R,0.0125,BR Tech Assigned
3,0000036/2,170.38750,T,0.0125,BR Tech Assigned
4,0000042/1,157.02500,T,0.0250,CSR Marina
...,...,...,...,...,...
159903,ES0089008/1,2106.40625,T,4.3000,Satellite (Non Fixed Satellite Earth Station)
159904,ES0089008/1,2216.00000,R,1.7600,Satellite (Non Fixed Satellite Earth Station)
159905,ES0089008/1,2216.00000,R,2.7300,Satellite (Non Fixed Satellite Earth Station)
159906,ES0089008/1,2232.50000,R,4.1400,Satellite (Non Fixed Satellite Earth Station)
